In [44]:
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# ch.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]


In [45]:
from pulp import *
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/ilyankou/passport-index-dataset/master/passport-index-matrix.csv", index_col = 0)

In [46]:
data

,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United States,Uruguay,Uzbekistan,Vanuatu,Vatican,Venezuela,Viet Nam,Yemen,Zambia,Zimbabwe
Passport,,,,,,,,,,,,,,,,,,,,,
Afghanistan,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Albania,0,-1,0,3,0,3,0,3,0,3,...,0,0,1,0,3,0,0,0,1,1
Algeria,0,0,-1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
Andorra,0,3,0,-1,0,3,3,3,2,3,...,2,3,3,3,3,3,1,0,1,1
Angola,0,0,0,0,-1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,0,3,0,3,1,3,3,1,0,3,...,0,3,1,0,3,-1,1,0,1,1
Viet Nam,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,-1,0,1,0
Yemen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1,0,0


In [47]:
data = data.replace([-1],3)
data = data.replace([1,2],0)
data = data.replace([3],1)

In [48]:
countries = data.index.values.tolist()

In [49]:
# Create model
m = LpProblem("Passport", LpMinimize)

# Variables
x = LpVariable.dicts('x', countries, lowBound=0, upBound=1, cat=LpInteger)

# Objective
m += sum(x[i] for i in countries)

# Constraint
for dest in countries:
    m += LpAffineExpression(list(map(tuple,(zip(x.values(),data[dest]))))) >= 1 #this approach is a little cryptic
    #m += sum(x[origin] * data.loc[origin][dest] for origin in countries) >= 1 #this approach is much slower

In [58]:
# Optimize

m.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[m.status])

Status = Optimal


In [13]:
# Print the value of the variables at the optimum
for i in countries:
    if x[i].varValue>0:
        print("%s = %f" % (x[i].name, x[i].varValue))

# Print the value of the objective
print("Objective = %f" % value(m.objective))

x_Afghanistan = 1.000000
x_Comoros = 1.000000
x_Czech_Republic = 1.000000
x_Djibouti = 1.000000
x_Gabon = 1.000000
x_Georgia = 1.000000
x_Hong_Kong = 1.000000
x_Iraq = 1.000000
x_Libya = 1.000000
x_Madagascar = 1.000000
x_Malaysia = 1.000000
x_Maldives = 1.000000
x_Nepal = 1.000000
x_New_Zealand = 1.000000
x_Niger = 1.000000
x_North_Korea = 1.000000
x_Papua_New_Guinea = 1.000000
x_Seychelles = 1.000000
x_Somalia = 1.000000
x_Uganda = 1.000000
x_United_Arab_Emirates = 1.000000
x_United_States = 1.000000
x_Zimbabwe = 1.000000
Objective = 23.000000
